In [ ]:
spark.streams.active

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
userSchema = StructType() \
        .add("window",StructType()\
             .add("start",TimestampType())\
             .add("end",TimestampType()))\
        .add("subject", StringType())\
        .add("count", LongType())

In [ ]:
raw_data = spark\
.readStream.format("parquet")\
.schema(userSchema)\
.load("/cms/users/carizapo/ming/groupdata");
raw_data.printSchema()

In [ ]:
raw_data_flow.lastProgress()

In [ ]:
raw_data_flow = raw_data.writeStream.queryName("hdfs").outputMode("Append").format("memory").start()

In [ ]:
raw_data_flow.stop()

In [ ]:
alerts = spark.sql("select * from hdfs")
alerts.show()